In [1]:
import pandas as pd
df_oldversion = pd.read_csv("./train.csv")
df = pd.read_csv("./train_encodedv.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 185 entries, Id to SaleCondition_Partial
dtypes: bool(122), float64(3), int64(34), object(26)
memory usage: 892.7+ KB


In [36]:
df.columns[df.columns.str.startswith("Alley")]

Index(['Alley_Grvl', 'Alley_Pave'], dtype='object')

In [41]:
df_oldversion[df_oldversion["Alley"].notna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
21,22,45,RM,57.0,7449,Pave,Grvl,Reg,Bnk,AllPub,...,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal,139400
30,31,70,C (all),50.0,8500,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,40000
56,57,160,FV,24.0,2645,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2009,WD,Abnorml,172500
79,80,50,RM,60.0,10440,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2009,WD,Normal,110000
87,88,160,FV,40.0,3951,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2009,New,Partial,164500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1405,50,RL,60.0,10410,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,1,2006,WD,Family,105000
1414,1415,50,RL,64.0,13053,Pave,Pave,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,207000
1427,1428,50,RL,60.0,10930,Pave,Grvl,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Normal,140000
1432,1433,30,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,64500


In [46]:
# I erroneously one-hot encoded Alley column. Most rows NaN. It needs indic var, interaction term instead
# To correct it I will delete the on-hot encoded columns, then add original column back on
df.drop(["Alley_Grvl", "Alley_Pave"], axis=1)
df["Alley"] = df_oldversion["Alley"]

In [55]:
df_oldversion["Alley"].value_counts(), "" , df["Alley"].value_counts()

(Alley
 Grvl    50
 Pave    41
 Name: count, dtype: int64,
 '',
 Alley
 Grvl    50
 Pave    41
 Name: count, dtype: int64)

In [65]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["LotShape_le"] = le.fit_transform(df["LotShape"])

In [67]:
# no, need to do it again -- reorder it. Reg at one end of the scale, IR3 (not IR1!) at the other end
df[["LotShape", "LotShape_le"]]

,LotShape,LotShape_le
0,Reg,3
1,Reg,3
2,IR1,0
3,IR1,0
4,IR1,0
...,...,...
1455,Reg,3
1456,Reg,3
1457,Reg,3
1458,Reg,3
